In [1]:
import numpy as np
import os
from os.path import join as pjoin
from glob import iglob
from shutil import copyfile

In [2]:
root_path = '/data/ksb'
bert_root = pjoin(root_path, 'BertSum/kor_PreSumm')
kobart_root = pjoin(root_path, 'BART/KoBART-summarization')
refsum_root = pjoin(root_path, 'RefSum/Refactoring-Summarization')

In [3]:
data_root = pjoin(root_path, 'korean_document')

In [4]:
kobert_root = pjoin(root_path, 'BertSum/koBertSum')
kobert_data = pjoin(kobert_root, 'ko_raw_stories')
kobert_data

'/data/ksb/BertSum/koBertSum/ko_raw_stories'

In [5]:
def mkdir_p(path):
    import errno
    try:
        os.makedirs(path)
    except OSError as err:
        if err.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
            
def del_folder(path):
    try:
        rmtree(path)
    except:
        pass

In [6]:
def get_article_list(path):
    
    article_list = []
    
    with jsonlines.open(path) as reader:
        for news in reader:
            content = news['article_original']
            idx = news['id']
            abs_summ = news['abstractive']
            ext_summ = news['extractive']

            article = (idx, content, abs_summ, ext_summ)
            article_list.append(article)
        
    return article_list
    

In [7]:
import jsonlines

train_data = pjoin(data_root, 'train.jsonl')
val_data = pjoin(data_root, 'vaild.jsonl')

train_article_list = get_article_list(train_data)
val_article_list = get_article_list(val_data)

(len(train_article_list), len(val_article_list))

(260697, 10000)

### BERT Extractor

### korBART Finetuning data


In [8]:
def copy_file(src_regex, dst_path):
    for src in iglob(src_regex, recursive=False):
        copyfile(src, dst_path)

In [9]:
kor_bart_data_dir = pjoin(kobart_root, 'data')

train_data_path = pjoin(kor_bart_data_dir, 'train.tsv')
test_data_path = pjoin(kor_bart_data_dir, 'test.tsv')

In [11]:
from kobart import get_kobart_tokenizer
kobart_tokenizer = get_kobart_tokenizer()

train_dataset = []
for data in train_article_list:
    news_id = data[0]
    news = data[1]
    summary = data[2]
    
    news_token = []
    for sent in news:
        news_token += kobart_tokenizer.tokenize(sent)
        
    summary_token = kobart_tokenizer.tokenize(summary)
    
    train_dataset.append((news_id, news, summary, news_token, summary_token))


using cached model


In [12]:
kobart_tokenizer = get_kobart_tokenizer()

val_dataset = []
for data in val_article_list:
    news_id = data[0]
    news = data[1]
    summary = data[2]
    
    news_token = []
    for sent in news:
        news_token += kobart_tokenizer.tokenize(sent)
        
    summary_token = kobart_tokenizer.tokenize(summary)
    
    val_dataset.append((news_id, news, summary, news_token, summary_token))

len(val_dataset)

using cached model


10000

In [13]:
(len(train_dataset), len(val_dataset))

(260697, 10000)

In [14]:
dataset = train_dataset + val_dataset
len(dataset)

270697

In [15]:
article_token_dist = [len(data[-2]) for data in dataset]
summary_token_dist = [len(data[-1]) for data in dataset]

token 분포에 따라 데이터 정제  

In [26]:
refined_dataset = [data for data in dataset if len(data[-2]) >= 377 and len(data[-2]) <= 421]

In [140]:
valid_dataset = [data for data in dataset if (len(data[-2]) >= 369 and len(data[-2]) < 376) or (len(data[-2]) > 421 and len(data[-2]) < 428)]
len(valid_dataset)

10492

In [27]:
import pandas as pd
refined_dataframe = pd.DataFrame(columns=['news_id', 'news', 'summary', 'news_token', 'summary_token'])

for data in refined_dataset:
    dic = {'news_id' : data[0],
           'news' : data[1],
           'summary' : data[2],
           'news_token' : data[3],
           'summary_token' : data[4]
          }
    refined_dataframe = refined_dataframe.append(dic, ignore_index=True)
    

In [28]:
refined_dataframe

,news_id,news,summary,news_token,summary_token
0,361192376,[대유위니아그룹의 위니아딤채가 발효기술과 음식 별 맞춤보관 기능이 돋보이는 2020...,위니아딤채는 '우리집 딤채를 소개합니다'라는 컨셉으로 제작한 2020년형 김치냉장고...,"[▁대, 유, 위, 니아, 그룹의, ▁위, 니아, 딤, 채, 가, ▁발효, 기술, ...","[▁위, 니아, 딤, 채, 는, ▁'우리, 집, ▁, 딤, 채를, ▁소개, 합, 니..."
1,330403000,"[라온 프라이빗타운 조성사업 조감도(자료사진)., 관광지 개발사업 추진과정에서 환경...",제주도에서 관광지 개발중에 이식한 곰솔 2000그루가 10년만에 고사하였는데 제주도...,"[▁라, 온, ▁프라이, 빗, 타운, ▁조성, 사업, ▁조, 감도, (, 자료, 사...","[▁제주, 도에서, ▁관광, 지, ▁개발, 중에, ▁이, 식, 한, ▁곰, 솔, ▁..."
2,338460499,[한국전통문화전당서 봉입식 유네스코 음식창의도시인 전주음식의 자료를 전주한지에 작성...,전주 음식을 보존·계승하기 위해서 전주지역을 대표하는 맛집과 향토음식점 등 101개...,"[▁한국, 전, 통, 문화, 전, 당, 서, ▁봉, 입, 식, ▁유, 네, 스코, ...","[▁전주, ▁음식을, ▁보존, ·, 계, 승, 하기, ▁위해서, ▁전주, 지역을, ..."
3,365584278,[외국인이 2주 연속 '사자'세를 이어갔으나 순매수 규모는 전 주에 비해 큰 폭으로...,외국인이 2주 연속 '사자'세를 이어가며 지난 주 가장 많이 사들이 종목은 반도체 ...,"[▁외국인이, ▁2주, ▁연속, ▁'사, 자', 세를, ▁이어, 갔, 으나, ▁순매...","[▁외국인이, ▁2주, ▁연속, ▁'사, 자', 세를, ▁이어, 가며, ▁지난, ▁..."
4,361447259,[27일 서울 종로구 KT스퀘어에서 열린 제3회 '시니어 토크콘서트'에서 황석영 작...,27일 서울 종로구에서 웰다잉시민운동 주체로 '시니어 토크콘서트'에 참여한 황석영 ...,"[▁27일, ▁서울, ▁종로구, ▁KT, 스, 퀘, 어, 에서, ▁열린, ▁제3, ...","[▁27일, ▁서울, ▁종로, 구에서, ▁웰, 다, 잉, 시민, 운동, ▁주체, 로..."
...,...,...,...,...,...
35973,333638470,"[환경부 “2개월 내에 평택항서 반출”, 평택항에 반입된 필리핀 수출폐기물을 둘러싸...",필리핀 수출폐기물이 평택항으로 반입되는 등 폐기물 관련 문제가 발생하자 평택시민들의...,"[▁환경부, ▁“, 2, 개월, ▁내에, ▁평택, 항, 서, ▁반, 출, ”, ▁평...","[▁필리핀, ▁수출, 폐, 기, 물이, ▁평택, 항, 으로, ▁반, 입, 되는, ▁..."
35974,355406915,"[제주삼다수 마스터스에 출전하는 박인비와 고진영, 최혜진 등 국내외 탑랭커들이 꿈나...",제주삼다수 마스터스에 출전하는 박인비와 고진영 등 세게 탑랭커들이 16명의 골프 꿈...,"[▁제주, 삼, 다, 수, ▁마스터, 스에, ▁출전, 하는, ▁박인, 비와, ▁고,...","[▁제주, 삼, 다, 수, ▁마스터, 스에, ▁출전, 하는, ▁박인, 비와, ▁고,..."
35975,361877342,"[서구청 전경., 대구 서구청이 지역 맞춤형 인재 양성을 통한 다양한 일자리 창출로...",대전 서구청이 올해 5억4천만 원을 투입하여 ‘청년고용 젊은기업 지원사업’ 등 지역...,"[▁서, 구청, ▁전경, ., ▁대구, ▁서, 구청, 이, ▁지역, ▁맞춤형, ▁인...","[▁대전, ▁서, 구청, 이, ▁올해, ▁5억, 4천, 만, ▁원을, ▁투입, 하여..."
35976,345268006,[직권남용·공직선거법 위반 혐의로 기소된 이재명 경기지사의 정치적 명운을 가를 법원...,공직사회도 재판결과를 예의주시하고 있는 이 지사의 사건담당재판부인 수원지법 성남지원...,"[▁직, 권, 남, 용, ·공, 직, 선거, 법, ▁위반, ▁혐의로, ▁기소된, ▁...","[▁공직, 사회, 도, ▁재판, 결과를, ▁예의, 주시, 하고, ▁있는, ▁이, ▁..."


In [141]:
import pandas as pd
valid_dataframe = pd.DataFrame(columns=['news_id', 'news', 'summary', 'news_token', 'summary_token'])

for data in valid_dataset:
    dic = {'news_id' : data[0],
           'news' : data[1],
           'summary' : data[2],
           'news_token' : data[3],
           'summary_token' : data[4]
          }
    valid_dataframe = valid_dataframe.append(dic, ignore_index=True)
    

In [142]:
valid_dataframe

,news_id,news,summary,news_token,summary_token
0,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[▁지난, ▁2004년, ▁시작해, ▁조선, 대, ▁학생들의, ▁대표적인, ▁행사,...","[▁‘, 조선, 대의, ▁새로운, ▁비, 상을, ▁꿈꾸, 다, ’를, ▁슬로, 건으..."
1,349628697,"[집배원이 또 사망했다., 올해만 과로사 등 9번째다., 우정사업본부 노사에 따르면...","19일 아침 충남 당진우체국 집배원이 또 사망, 올해에만 9번째 과로사가 발생한 가...","[▁집, 배, 원이, ▁또, ▁사망, 했다., ▁올해, 만, ▁과, 로, 사, ▁등...","[▁19일, ▁아침, ▁충남, ▁당, 진, 우, 체, 국, ▁집, 배, 원이, ▁또..."
2,330964970,"[더불어민주당 4선 중진인 송영길 의원이 ""신한울 3·4호기 건설 재개를 검토해야 ...","송영길의원은 신한울 3.4호기 문제가 공식 의제로 논의된 적이 없는 점을 비판하고,...","[▁더불어민주당, ▁4, 선, ▁중, 진, 인, ▁송, 영, 길, ▁의원이, ▁"",...","[▁송, 영, 길, 의, 원은, ▁신한, 울, ▁3, .4, 호, 기, ▁문제가, ..."
3,350650390,"[직장인 희망하는 점심시간 ‘1시간 30분’, 직장인이 생각하는 이상적인 점심시간은...",취업포털 잡코리아가 남녀직장인을 대상으로 ‘점심시간 활용’에 대한 설문조사를 진행한...,"[▁직장인, ▁희망, 하는, ▁점심, 시간, ▁‘, 1, 시간, ▁30분, ’, ▁...","[▁취업, 포, 털, ▁잡, 코리아, 가, ▁남녀, 직, 장, 인을, ▁대상으로, ..."
4,336495657,[[청주]청주시는 6일 청주 예술의 전당 소공연장에서 시정발전연구모임 '놀일터연구소...,청주시는 청주 예술의 전당 소공연장에서 시정발전연구모임 '놀일터연구소' 오리엔테이션...,"[▁[, 청, 주, ], 청, 주, 시는, ▁6일, ▁청주, ▁예술의, ▁전당, ▁...","[▁청주, 시는, ▁청주, ▁예술의, ▁전당, ▁소, 공연, 장에서, ▁시정, 발전..."
...,...,...,...,...,...
10487,339736616,"[[서울경제] 서울 주택 매매가가 3개월 연속 하락했다., 대출 규제 등의 영향으로...","서울 주택 매매가는 올 1월부터 3개월 연속 마이너스 변동률을 기록하고 있는데, 세...","[▁[, 서울, 경제, ], ▁서울, ▁주택, ▁매매, 가가, ▁3개월, ▁연속, ...","[▁서울, ▁주택, ▁매매, 가는, ▁올, ▁1월부터, ▁3개월, ▁연속, ▁마이너..."
10488,338582154,[대구를 대표하는 원로 경제인인 이인중(74) 화성산업 대표이사 회장이 경영 일선에...,대구를 대표하는 원로 경제인인 이인중(74) 화성산업 대표이사 회장이 이홍중 대표이...,"[▁대구, 를, ▁대표하는, ▁원로, ▁경제, 인인, ▁이인, 중, (7, 4), ...","[▁대구, 를, ▁대표하는, ▁원로, ▁경제, 인인, ▁이인, 중, (7, 4), ..."
10489,330584592,"[임재업 지역경제 활성화 버팀목 역할 충실하겠다., (동양일보 임재업 기자) 정붕익...","청주산단관리공단이 지역경제활성화 역할에 충실을 다하고 있으며, 경쟁력 강화사업에 역...","[▁임, 재, 업, ▁지역경제, ▁활성화, ▁버, 팀, 목, ▁역할, ▁충실, 하겠...","[▁청주, 산, 단, 관리, 공, 단이, ▁지역경제, 활, 성화, ▁역할, 에, ▁..."
10490,361787219,[KT는 현대건설과 1일 계동 현대건설 사옥에서 '5G 기반 스마트 건설자동화 기술...,KT는 현대건설과 1일 계동 현대건설 사옥에서 '5G 기반 스마트 건설자동화 기술개...,"[▁KT는, ▁현대, 건설, 과, ▁1일, ▁계, 동, ▁현대, 건설, ▁사, 옥,...","[▁KT는, ▁현대, 건설, 과, ▁1일, ▁계, 동, ▁현대, 건설, ▁사, 옥,..."


In [172]:
kobart_test_path = pjoin(kobart_root, 'model_test')
mkdir_p(kobart_test_path)

valid_dataframe.to_csv(pjoin(kobart_test_path, 'test.csv'), sep='\t', index=False)

In [29]:
refined_train_dataset = refined_dataframe[:30000]

In [30]:
refined_val_dataset = refined_dataframe[30000:]
refined_val_dataset

,news_id,news,summary,news_token,summary_token
30000,353481673,[한국가스공사는 지난 19일 인천 서구청 등 5개 기관과 LNG 청소차·믹서트럭 개...,한국가스공사는 19일 인천 서구청 등 5개 기관과 액화천연가스(LNG) 차량 도입 ...,"[▁한국, 가스, 공사는, ▁지난, ▁19일, ▁인천, ▁서, 구청, ▁등, ▁5개...","[▁한국, 가스, 공사는, ▁19일, ▁인천, ▁서, 구청, ▁등, ▁5개, ▁기관..."
30001,340654075,"[8일 오전 울산시청과 5개 구·군청, 동사무소 등의 인터넷 연결이 한때 끊어져 울...",8일 오전 10시 22분 즈음 울산시 시청 본관 무정전 전원 장치(UPS)에 화재가...,"[▁8일, ▁오전, ▁울산, 시청, 과, ▁5개, ▁구, ·, 군, 청,, ▁동, ...","[▁8일, ▁오전, ▁10시, ▁22, 분, ▁즈, 음, ▁울산, 시, ▁시청, ▁..."
30002,367035292,[예산군이 17개 주요관광지점 방문객 수를 지난해 10월 대비 누적통계 수치를 파악...,예산군이 17개 주요관광지점 방문객 수를 지난해 10월 대비 누적통계 수치로 분석한...,"[▁예산, 군이, ▁17, 개, ▁주요, 관광, 지점, ▁방문, 객, ▁수를, ▁지...","[▁예산, 군이, ▁17, 개, ▁주요, 관광, 지점, ▁방문, 객, ▁수를, ▁지..."
30003,358939692,[지역 국회의원이 최근 일본 수산물을 싣고 부산항에 들어와 국내 도로를 활보하는 일...,국회 농림축산식품해양수산위원회 소속 더불어민주당 윤준호의원은 5일 국회 정론관에서 ...,"[▁지역, ▁국회의, 원이, ▁최근, ▁일본, ▁수산, 물을, ▁싣, 고, ▁부산,...","[▁국회, ▁농림, 축, 산, 식품, 해양, 수산, 위원회, ▁소속, ▁더불어민주당..."
30004,353782536,"[올해로 3회째 제주에서 개최되는 PGA 투어의 성공을 위해 제주도와 제주관광공사,...",제주도가 '제3회 더 CJ컵 앳 나인브릿지' PGA 투어를 통해 제주 홍보를 극대화...,"[▁올해로, ▁3회, 째, ▁제주, 에서, ▁개최, 되는, ▁P, G, A, ▁투,...","[▁제주, 도가, ▁'제, 3, 회, ▁더, ▁CJ, 컵, ▁, 앳, ▁나, 인, ..."
...,...,...,...,...,...
35973,333638470,"[환경부 “2개월 내에 평택항서 반출”, 평택항에 반입된 필리핀 수출폐기물을 둘러싸...",필리핀 수출폐기물이 평택항으로 반입되는 등 폐기물 관련 문제가 발생하자 평택시민들의...,"[▁환경부, ▁“, 2, 개월, ▁내에, ▁평택, 항, 서, ▁반, 출, ”, ▁평...","[▁필리핀, ▁수출, 폐, 기, 물이, ▁평택, 항, 으로, ▁반, 입, 되는, ▁..."
35974,355406915,"[제주삼다수 마스터스에 출전하는 박인비와 고진영, 최혜진 등 국내외 탑랭커들이 꿈나...",제주삼다수 마스터스에 출전하는 박인비와 고진영 등 세게 탑랭커들이 16명의 골프 꿈...,"[▁제주, 삼, 다, 수, ▁마스터, 스에, ▁출전, 하는, ▁박인, 비와, ▁고,...","[▁제주, 삼, 다, 수, ▁마스터, 스에, ▁출전, 하는, ▁박인, 비와, ▁고,..."
35975,361877342,"[서구청 전경., 대구 서구청이 지역 맞춤형 인재 양성을 통한 다양한 일자리 창출로...",대전 서구청이 올해 5억4천만 원을 투입하여 ‘청년고용 젊은기업 지원사업’ 등 지역...,"[▁서, 구청, ▁전경, ., ▁대구, ▁서, 구청, 이, ▁지역, ▁맞춤형, ▁인...","[▁대전, ▁서, 구청, 이, ▁올해, ▁5억, 4천, 만, ▁원을, ▁투입, 하여..."
35976,345268006,[직권남용·공직선거법 위반 혐의로 기소된 이재명 경기지사의 정치적 명운을 가를 법원...,공직사회도 재판결과를 예의주시하고 있는 이 지사의 사건담당재판부인 수원지법 성남지원...,"[▁직, 권, 남, 용, ·공, 직, 선거, 법, ▁위반, ▁혐의로, ▁기소된, ▁...","[▁공직, 사회, 도, ▁재판, 결과를, ▁예의, 주시, 하고, ▁있는, ▁이, ▁..."


In [32]:
import pickle

train_idx_filepath = pjoin(kor_bart_data_dir, 'train_data_idx.txt')
test_idx_filepath = pjoin(kor_bart_data_dir, 'test_data_idx.txt')

print(train_idx_filepath)
with open(train_idx_filepath, 'wb') as f:
    pickle.dump([data[0] for _, data in refined_train_dataset.iterrows()], f)
    
print(test_idx_filepath)
with open(test_idx_filepath, 'wb') as f:
    pickle.dump([data[0] for _, data in refined_val_dataset.iterrows()], f)

/data/ksb/BART/KoBART-summarization/data/train_data_idx.txt
/data/ksb/BART/KoBART-summarization/data/test_data_idx.txt


In [33]:
train_idx_filepath = pjoin(kor_bart_data_dir, 'train_data_idx.txt')
valid_idx_filepath = pjoin(kor_bart_data_dir, 'val_data_idx.txt')

In [34]:
import pickle
with open(train_idx_filepath, 'rb') as f:
    tmp = pickle.load(f)
len(tmp)

30000

In [35]:
save_train_data = refined_train_dataset.drop(['news_id', 'news_token', 'summary_token'], axis=1)
save_train_data

,news,summary
0,[대유위니아그룹의 위니아딤채가 발효기술과 음식 별 맞춤보관 기능이 돋보이는 2020...,위니아딤채는 '우리집 딤채를 소개합니다'라는 컨셉으로 제작한 2020년형 김치냉장고...
1,"[라온 프라이빗타운 조성사업 조감도(자료사진)., 관광지 개발사업 추진과정에서 환경...",제주도에서 관광지 개발중에 이식한 곰솔 2000그루가 10년만에 고사하였는데 제주도...
2,[한국전통문화전당서 봉입식 유네스코 음식창의도시인 전주음식의 자료를 전주한지에 작성...,전주 음식을 보존·계승하기 위해서 전주지역을 대표하는 맛집과 향토음식점 등 101개...
3,[외국인이 2주 연속 '사자'세를 이어갔으나 순매수 규모는 전 주에 비해 큰 폭으로...,외국인이 2주 연속 '사자'세를 이어가며 지난 주 가장 많이 사들이 종목은 반도체 ...
4,[27일 서울 종로구 KT스퀘어에서 열린 제3회 '시니어 토크콘서트'에서 황석영 작...,27일 서울 종로구에서 웰다잉시민운동 주체로 '시니어 토크콘서트'에 참여한 황석영 ...
...,...,...
29995,"[이엠코리아가 아프리카돼지열병(ASF) 대책의 직접적인 수혜주로 부상할 전망이다.,...",정부가 아프리카돼지열병 대응 조치로 양돈가에 잔반 급여를 전면 금지토록 하면서 유기...
29996,"[동명대학교 창업교육센터, 제 5회 학생 창업 국제컨퍼런스&교류회 운영을 위한 실무...",지난 22일 동명대 산학협력관에서 동명대 경남대 등 총 10개 대학이 참석한 가운데...
29997,"[경북 포항시는 아동이 건강하고 안전하게 성장 할 수 있도록 지원하고, 세상에서 가...",경북 포항시는 18세 미만 아동의 목소리가 반영되는 지역사회이자 유엔 아동권리협약의...
29998,"[사업용 덤프트럭과 콘크리트믹서트럭, 콘크리트펌프에 대한 신규 등록이 2년간 제한된...",23일 국토교통부는 건설기계 수급조절위원회’를 열어 영세한 건설기계 대여사업자 보호...


In [36]:
kor_bart_data_dir = pjoin(kobart_root, 'data')

train_data_path = pjoin(kor_bart_data_dir, 'train.tsv')
test_data_path = pjoin(kor_bart_data_dir, 'test.tsv')

In [37]:
save_train_data.to_csv(train_data_path, columns=['news','summary'], sep='\t', index=False)

In [38]:
save_val_data = refined_val_dataset.drop(['news_id', 'news_token', 'summary_token'], axis=1)
save_val_data

,news,summary
30000,[한국가스공사는 지난 19일 인천 서구청 등 5개 기관과 LNG 청소차·믹서트럭 개...,한국가스공사는 19일 인천 서구청 등 5개 기관과 액화천연가스(LNG) 차량 도입 ...
30001,"[8일 오전 울산시청과 5개 구·군청, 동사무소 등의 인터넷 연결이 한때 끊어져 울...",8일 오전 10시 22분 즈음 울산시 시청 본관 무정전 전원 장치(UPS)에 화재가...
30002,[예산군이 17개 주요관광지점 방문객 수를 지난해 10월 대비 누적통계 수치를 파악...,예산군이 17개 주요관광지점 방문객 수를 지난해 10월 대비 누적통계 수치로 분석한...
30003,[지역 국회의원이 최근 일본 수산물을 싣고 부산항에 들어와 국내 도로를 활보하는 일...,국회 농림축산식품해양수산위원회 소속 더불어민주당 윤준호의원은 5일 국회 정론관에서 ...
30004,"[올해로 3회째 제주에서 개최되는 PGA 투어의 성공을 위해 제주도와 제주관광공사,...",제주도가 '제3회 더 CJ컵 앳 나인브릿지' PGA 투어를 통해 제주 홍보를 극대화...
...,...,...
35973,"[환경부 “2개월 내에 평택항서 반출”, 평택항에 반입된 필리핀 수출폐기물을 둘러싸...",필리핀 수출폐기물이 평택항으로 반입되는 등 폐기물 관련 문제가 발생하자 평택시민들의...
35974,"[제주삼다수 마스터스에 출전하는 박인비와 고진영, 최혜진 등 국내외 탑랭커들이 꿈나...",제주삼다수 마스터스에 출전하는 박인비와 고진영 등 세게 탑랭커들이 16명의 골프 꿈...
35975,"[서구청 전경., 대구 서구청이 지역 맞춤형 인재 양성을 통한 다양한 일자리 창출로...",대전 서구청이 올해 5억4천만 원을 투입하여 ‘청년고용 젊은기업 지원사업’ 등 지역...
35976,[직권남용·공직선거법 위반 혐의로 기소된 이재명 경기지사의 정치적 명운을 가를 법원...,공직사회도 재판결과를 예의주시하고 있는 이 지사의 사건담당재판부인 수원지법 성남지원...


In [39]:
save_val_data.to_csv(test_data_path, columns=['news','summary'], sep='\t', index=False)

### After Training koBART

In [7]:
import pandas as pd
from ast import literal_eval

kobart_test_path = pjoin(kobart_root, 'model_test')
valid_dataframe = pd.read_csv(pjoin(kobart_test_path, 'test.csv'), 
                              converters={"news" : literal_eval, "news_token" : literal_eval,"summary_token" : literal_eval}, 
                              sep='\t')

valid_dataframe.iloc[0][1]

['지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대장정’이 지난 5일 해단식을 끝으로 일정을 마감했다.',
 '조선대는 지난 5일 오후 서석홀 4층 대호전기홀에서 ‘2019학년도 조선대학교 제16기 국토대장정’ 해단식을 열었다.',
 '국토대장정은 매년 여름방학에 조선대 학생들이 직접 우리 국토를 걸으면서 애국심을 느끼고, 단체생활을 통해 협동심과 리더십, 공동체 의식을 함양할 수 있도록 지원하는 행사다.',
 '해마다 학생들에게 큰 호응을 받아 왔다.',
 '올해 국토대장정은 16번째 행사로 ‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행됐다.',
 '지난달 24일 출정식을 갖고 25일 새벽에 출발한 52명의 대원들은 총 10박 11일간 제주도 일대 총 321㎞를 행군했다.',
 '이는 지난해보다 더 늘어난 행군 기록이다.',
 '대원들은 목포항을 거쳐 배편으로 제주도로 이동해 관음사 야영지로 이동했다.',
 '대원들은 다음날인 지난달 26일 제주 애월읍의 곽지해수욕장, 지난달 27일 서귀포 하모해수욕장, 지난달 28일 서귀포 체육공원을 향해 걸었다.',
 '이어 지난달 29일 서귀포 표선해수욕장을 거쳐 지난달 30일 서귀포 섭지코지와 성산일출봉을 여행했다.',
 '이달 1일에는 한라산 백록담을 등반했다.',
 '그 다음날에는 제주 함덕해수욕장에 도착해 시간을 보냈다.',
 '이후 3일 휴식시간을 거져 4일 제주항에서 배를 타고 목포항에 도착했다.',
 '마침내 지난 5일 대원들은 동신대에서 모교인 조선대로 걸어서 귀교해 일정을 끝마쳤다.',
 '대원들은 대장정 기간 모든 야영지에서 쓰레기를 수거하는 등 환경정화활동을 펼치기도 했다.',
 '5일 열린 해단식에선 참가자 전원이 완주에 성공한 기념으로 국토대장정 대장을 맡은 김준연(정치외교학과 4학년) 학생 등을 포함한 조장단에게 표창장이 수여됐다.']

In [8]:
valid_dataframe

,news_id,news,summary,news_token,summary_token
0,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[▁지난, ▁2004년, ▁시작해, ▁조선, 대, ▁학생들의, ▁대표적인, ▁행사,...","[▁‘, 조선, 대의, ▁새로운, ▁비, 상을, ▁꿈꾸, 다, ’를, ▁슬로, 건으..."
1,349628697,"[집배원이 또 사망했다., 올해만 과로사 등 9번째다., 우정사업본부 노사에 따르면...","19일 아침 충남 당진우체국 집배원이 또 사망, 올해에만 9번째 과로사가 발생한 가...","[▁집, 배, 원이, ▁또, ▁사망, 했다., ▁올해, 만, ▁과, 로, 사, ▁등...","[▁19일, ▁아침, ▁충남, ▁당, 진, 우, 체, 국, ▁집, 배, 원이, ▁또..."
2,330964970,"[더불어민주당 4선 중진인 송영길 의원이 ""신한울 3·4호기 건설 재개를 검토해야 ...","송영길의원은 신한울 3.4호기 문제가 공식 의제로 논의된 적이 없는 점을 비판하고,...","[▁더불어민주당, ▁4, 선, ▁중, 진, 인, ▁송, 영, 길, ▁의원이, ▁"",...","[▁송, 영, 길, 의, 원은, ▁신한, 울, ▁3, .4, 호, 기, ▁문제가, ..."
3,350650390,"[직장인 희망하는 점심시간 ‘1시간 30분’, 직장인이 생각하는 이상적인 점심시간은...",취업포털 잡코리아가 남녀직장인을 대상으로 ‘점심시간 활용’에 대한 설문조사를 진행한...,"[▁직장인, ▁희망, 하는, ▁점심, 시간, ▁‘, 1, 시간, ▁30분, ’, ▁...","[▁취업, 포, 털, ▁잡, 코리아, 가, ▁남녀, 직, 장, 인을, ▁대상으로, ..."
4,336495657,[[청주]청주시는 6일 청주 예술의 전당 소공연장에서 시정발전연구모임 '놀일터연구소...,청주시는 청주 예술의 전당 소공연장에서 시정발전연구모임 '놀일터연구소' 오리엔테이션...,"[▁[, 청, 주, ], 청, 주, 시는, ▁6일, ▁청주, ▁예술의, ▁전당, ▁...","[▁청주, 시는, ▁청주, ▁예술의, ▁전당, ▁소, 공연, 장에서, ▁시정, 발전..."
...,...,...,...,...,...
10487,339736616,"[[서울경제] 서울 주택 매매가가 3개월 연속 하락했다., 대출 규제 등의 영향으로...","서울 주택 매매가는 올 1월부터 3개월 연속 마이너스 변동률을 기록하고 있는데, 세...","[▁[, 서울, 경제, ], ▁서울, ▁주택, ▁매매, 가가, ▁3개월, ▁연속, ...","[▁서울, ▁주택, ▁매매, 가는, ▁올, ▁1월부터, ▁3개월, ▁연속, ▁마이너..."
10488,338582154,[대구를 대표하는 원로 경제인인 이인중(74) 화성산업 대표이사 회장이 경영 일선에...,대구를 대표하는 원로 경제인인 이인중(74) 화성산업 대표이사 회장이 이홍중 대표이...,"[▁대구, 를, ▁대표하는, ▁원로, ▁경제, 인인, ▁이인, 중, (7, 4), ...","[▁대구, 를, ▁대표하는, ▁원로, ▁경제, 인인, ▁이인, 중, (7, 4), ..."
10489,330584592,"[임재업 지역경제 활성화 버팀목 역할 충실하겠다., (동양일보 임재업 기자) 정붕익...","청주산단관리공단이 지역경제활성화 역할에 충실을 다하고 있으며, 경쟁력 강화사업에 역...","[▁임, 재, 업, ▁지역경제, ▁활성화, ▁버, 팀, 목, ▁역할, ▁충실, 하겠...","[▁청주, 산, 단, 관리, 공, 단이, ▁지역경제, 활, 성화, ▁역할, 에, ▁..."
10490,361787219,[KT는 현대건설과 1일 계동 현대건설 사옥에서 '5G 기반 스마트 건설자동화 기술...,KT는 현대건설과 1일 계동 현대건설 사옥에서 '5G 기반 스마트 건설자동화 기술개...,"[▁KT는, ▁현대, 건설, 과, ▁1일, ▁계, 동, ▁현대, 건설, ▁사, 옥,...","[▁KT는, ▁현대, 건설, 과, ▁1일, ▁계, 동, ▁현대, 건설, ▁사, 옥,..."


In [9]:
from transformers.models.bart import BartForConditionalGeneration
from kobart import get_kobart_tokenizer
import torch

model = BartForConditionalGeneration.from_pretrained(pjoin(kobart_root, 'kobart_summary'))
tokenizer = get_kobart_tokenizer()

using cached model


In [10]:
from numpy import dot 
from numpy.linalg import norm 
import numpy as np        

def cos_sim(vec_a, vec_b): 
    return dot(vec_a, vec_b)/(norm(vec_a)*norm(vec_b))

In [29]:
import gensim
print(gensim.__version__)

from gensim.models import FastText
import fasttext.util
from gensim import models
from gensim.matutils import softcossim
from gensim import corpora
from gensim.utils import simple_preprocess
import gensim.downloader as api

from scipy import sparse, stats
from collections import deque


3.8.0


In [12]:
ft = api.load('fasttext-wiki-news-subwords-300')
ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

[=-------------------------------------------------] 2.4% 23.4/958.4MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 958.5/958.4MB downloaded


In [17]:
ft

In [18]:
ko_model

In [18]:
len(ko_model.wv.vocab)

2000000

In [49]:
from gensim import utils, matutils  
def get_similarity(w1, w2, model):
    return dot(matutils.unitvec(model[w1]), matutils.unitvec(model[w2]))

In [50]:
def similarity_matrix(model, dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100, dtype=np.float32):
    
#     print("constructing a term similarity matrix")
    matrix_order = len(dictionary)
    matrix_nonzero = [1] * matrix_order
    matrix = sparse.identity(matrix_order, dtype=dtype, format="dok")
    num_skipped = 0
    
    # Decide the order of rows.
    if tfidf is None:
        word_indices = deque(sorted(dictionary.keys()))
    else:
        assert max(tfidf.idfs) < matrix_order
        word_indices = deque([
            index for index, _
            in sorted(tfidf.idfs.items(), key=lambda x: (x[1], -x[0]), reverse=True)
        ])

    # Traverse rows.
    for row_number, w1_index in enumerate(list(word_indices)):
        word_indices.popleft()
#         if row_number % 1000 == 0:
#             print(
#                 "PROGRESS: at %.02f%% rows (%d / %d, %d skipped, %.06f%% density)",
#                 100.0 * (row_number + 1) / matrix_order, row_number + 1, matrix_order,
#                 num_skipped, 100.0 * matrix.getnnz() / matrix_order**2)
        w1 = dictionary[w1_index]
        if w1 not in model.wv.vocab:
            num_skipped += 1
            continue  # A word from the dictionary is not present in the word2vec model.

        # Traverse upper triangle columns.
        if matrix_order <= nonzero_limit + 1:  # Traverse all columns.
            columns = (
                (w2_index, get_similarity(w1, dictionary[w2_index], model))
                for w2_index in word_indices
                if dictionary[w2_index] in model.wv.vocab)
        else:  # Traverse only columns corresponding to the embeddings closest to w1.
            num_nonzero = matrix_nonzero[w1_index] - 1
            columns = (
                (dictionary.token2id[w2], similarity)
                for _, (w2, similarity)
                in zip(
                    range(nonzero_limit - num_nonzero),
                    model.similar_by_word(w1, nonzero_limit - num_nonzero)
                )
                if w2 in dictionary.token2id
            )
            columns = sorted(columns, key=lambda x: x[0])

        for w2_index, sim in columns:

            if sim > threshold and matrix_nonzero[w2_index] <= nonzero_limit:
                element = sim**exponent
                matrix[w1_index, w2_index] = element
                matrix_nonzero[w1_index] += 1
                matrix[w2_index, w1_index] = element
                matrix_nonzero[w2_index] += 1
#     print(
#         "constructed a term similarity matrix with %0.6f %% nonzero elements",
#         100.0 * matrix.getnnz() / matrix_order**2
#     )
    return matrix.tocsc()

In [51]:
class DocSimilarity:
    
    def __init__(self, model, documents, tfidf=None, expo=2.0, threshold=0.0, nonzero_limit=100):
        self.model = model
        self.dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])
        self.similarity_matrix = similarity_matrix(self.model, self.dictionary, tfidf=tfidf, 
                                                              threshold=threshold, exponent=expo, 
                                                              nonzero_limit=nonzero_limit)
    
    def get_similarity(self, src_doc, tgt_doc):
        src_sent = ' '.join(src_doc)
        tgt_sent = ' '.join(tgt_doc)

        src_bow = self.dictionary.doc2bow(simple_preprocess(src_sent))
        tgt_bow = self.dictionary.doc2bow(simple_preprocess(tgt_sent))
        
        sim = softcossim(src_bow, tgt_bow, self.similarity_matrix)
        return sim
        

In [ ]:
from rouge import Rouge

result_dataframe = pd.DataFrame(columns=['news_id', 'news', 'summary', 'news_token', 'summary_token', 'result', 'cos_sim'])

rouge = Rouge()
for data in valid_dataframe.iterrows():
    
    news = data[1][1]
    summ = data[1][2]
    
    similar = DocSimilarity(model=ko_model, documents=[' '.join(news)])
    
    input_ids = tokenizer.encode(' '.join(news))
    input_ids = torch.tensor(input_ids)
    input_ids = input_ids.unsqueeze(0)
    
    output = model.generate(input_ids, eos_token_id=1, max_length=512, num_beams=5)
    gen_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    
    rouge_score = rouge.get_scores(summ, gen_summary)[0]
    cos_sim = similar.get_similarity(news, [gen_summary])
    print(rouge_score, "\n")
    print(cos_sim, "\n\n")
    
    dic = {'news_id' : data[1][0],
           'news' : news,
           'summary' : summ,
           'news_token' : data[1][3],
           'summary_token' : data[1][4],
           'result': gen_summary,
           'cos_sim': cos_sim,
           'rouge_1_R' : rouge_score['rouge-1']['r'],
           'rouge_1_P' : rouge_score['rouge-1']['p'],
           'rouge_1_F' : rouge_score['rouge-1']['f'],
           'rouge_2_R' : rouge_score['rouge-2']['r'],
           'rouge_2_P' : rouge_score['rouge-2']['p'],
           'rouge_2_F' : rouge_score['rouge-2']['f'],
           'rouge_L_R' : rouge_score['rouge-l']['r'],
           'rouge_L_P' : rouge_score['rouge-l']['p'],
           'rouge_L_F' : rouge_score['rouge-l']['f']
          }
    result_dataframe = result_dataframe.append(dic, ignore_index=True)


/tmp/ipykernel_1810/3048794524.py:45: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  model.similar_by_word(w1, nonzero_limit - num_nonzero)
/tmp/ipykernel_1810/2357680832.py:17: DeprecationWarning: Call to deprecated `softcossim` (Function will be removed in 4.0.0, use gensim.similarities.termsim.SparseTermSimilarityMatrix.inner_product instead).
  sim = softcossim(src_bow, tgt_bow, self.similarity_matrix)


{'rouge-1': {'r': 0.30952380952380953, 'p': 0.65, 'f': 0.41935483433923}, 'rouge-2': {'r': 0.18181818181818182, 'p': 0.42105263157894735, 'f': 0.25396824975560595}, 'rouge-l': {'r': 0.21428571428571427, 'p': 0.45, 'f': 0.29032257627471386}} 

0.4542155977802483 


{'rouge-1': {'r': 0.125, 'p': 0.16, 'f': 0.14035087226839044}, 'rouge-2': {'r': 0.09375, 'p': 0.125, 'f': 0.10714285224489818}, 'rouge-l': {'r': 0.125, 'p': 0.16, 'f': 0.14035087226839044}} 

0.4095370889301985 


{'rouge-1': {'r': 0.058823529411764705, 'p': 0.08, 'f': 0.06779660528583777}, 'rouge-2': {'r': 0.027777777777777776, 'p': 0.04, 'f': 0.03278688040849306}, 'rouge-l': {'r': 0.058823529411764705, 'p': 0.08, 'f': 0.06779660528583777}} 

0.6084707279100039 


{'rouge-1': {'r': 0.44, 'p': 0.5238095238095238, 'f': 0.47826086460302464}, 'rouge-2': {'r': 0.16, 'p': 0.2, 'f': 0.17777777283950633}, 'rouge-l': {'r': 0.4, 'p': 0.47619047619047616, 'f': 0.4347826037334594}} 

0.5905072872329199 


{'rouge-1': {'r': 0.23684210526

/tmp/ipykernel_1810/2185470925.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  return dot(matutils.unitvec(model[w1]), matutils.unitvec(model[w2]))


{'rouge-1': {'r': 0.5128205128205128, 'p': 0.8, 'f': 0.6249999952392579}, 'rouge-2': {'r': 0.4318181818181818, 'p': 0.76, 'f': 0.5507246330602815}, 'rouge-l': {'r': 0.5128205128205128, 'p': 0.8, 'f': 0.6249999952392579}} 

0.8271100563055839 


{'rouge-1': {'r': 0.05813953488372093, 'p': 0.4166666666666667, 'f': 0.10204081417742612}, 'rouge-2': {'r': 0.03409090909090909, 'p': 0.2727272727272727, 'f': 0.06060605863075203}, 'rouge-l': {'r': 0.05813953488372093, 'p': 0.4166666666666667, 'f': 0.10204081417742612}} 

0.6440122463037 


{'rouge-1': {'r': 0.2857142857142857, 'p': 0.26666666666666666, 'f': 0.27586206397146257}, 'rouge-2': {'r': 0.1724137931034483, 'p': 0.1724137931034483, 'f': 0.17241378810344843}, 'rouge-l': {'r': 0.2857142857142857, 'p': 0.26666666666666666, 'f': 0.27586206397146257}} 

0.5181974230763756 


{'rouge-1': {'r': 0.6071428571428571, 'p': 0.6538461538461539, 'f': 0.6296296246364884}, 'rouge-2': {'r': 0.4482758620689655, 'p': 0.5, 'f': 0.4727272677421488}, 'rouge-

{'rouge-1': {'r': 0.057692307692307696, 'p': 0.13043478260869565, 'f': 0.07999999574755579}, 'rouge-2': {'r': 0.018518518518518517, 'p': 0.045454545454545456, 'f': 0.02631578536011145}, 'rouge-l': {'r': 0.057692307692307696, 'p': 0.13043478260869565, 'f': 0.07999999574755579}} 

0.6761225329245653 


{'rouge-1': {'r': 0.34210526315789475, 'p': 0.5416666666666666, 'f': 0.4193548339646202}, 'rouge-2': {'r': 0.16666666666666666, 'p': 0.30434782608695654, 'f': 0.21538461081183444}, 'rouge-l': {'r': 0.34210526315789475, 'p': 0.5416666666666666, 'f': 0.4193548339646202}} 

0.6949469587643144 


{'rouge-1': {'r': 0.3103448275862069, 'p': 0.3, 'f': 0.3050847407641483}, 'rouge-2': {'r': 0.1935483870967742, 'p': 0.20689655172413793, 'f': 0.19999999500555568}, 'rouge-l': {'r': 0.2413793103448276, 'p': 0.23333333333333334, 'f': 0.23728813059465684}} 

0.39514694438375425 


{'rouge-1': {'r': 0.125, 'p': 0.3333333333333333, 'f': 0.18181817785123974}, 'rouge-2': {'r': 0.024390243902439025, 'p': 0.07

...

{'rouge-1': {'r': 0.42857142857142855, 'p': 0.1875, 'f': 0.2608695609829868}, 'rouge-2': {'r': 0.2, 'p': 0.0967741935483871, 'f': 0.13043477821361074}, 'rouge-l': {'r': 0.35714285714285715, 'p': 0.15625, 'f': 0.2173913001134216}} 

0.28731204324314 


{'rouge-1': {'r': 0.13793103448275862, 'p': 0.2857142857142857, 'f': 0.18604650723634408}, 'rouge-2': {'r': 0.07142857142857142, 'p': 0.15384615384615385, 'f': 0.09756097127900079}, 'rouge-l': {'r': 0.13793103448275862, 'p': 0.2857142857142857, 'f': 0.18604650723634408}} 

0.4296020132513654 


{'rouge-1': {'r': 0.3283582089552239, 'p': 0.6875, 'f': 0.4444444400693807}, 'rouge-2': {'r': 0.19480519480519481, 'p': 0.45454545454545453, 'f': 0.27272726852727275}, 'rouge-l': {'r': 0.31343283582089554, 'p': 0.65625, 'f': 0.4242424198673605}} 

0.6506161723125011 


{'rouge-1': {'r': 0.047619047619047616, 'p': 0.045454545454545456, 'f': 0.04651162290968145}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.047619047619047616, 'p': 0

{'rouge-1': {'r': 0.14285714285714285, 'p': 0.21428571428571427, 'f': 0.17142856662857156}, 'rouge-2': {'r': 0.07317073170731707, 'p': 0.10714285714285714, 'f': 0.08695651691661443}, 'rouge-l': {'r': 0.14285714285714285, 'p': 0.21428571428571427, 'f': 0.17142856662857156}} 

0.5928046203621437 


{'rouge-1': {'r': 0.2765957446808511, 'p': 0.3939393939393939, 'f': 0.32499999515312505}, 'rouge-2': {'r': 0.1568627450980392, 'p': 0.24242424242424243, 'f': 0.19047618570578245}, 'rouge-l': {'r': 0.2127659574468085, 'p': 0.30303030303030304, 'f': 0.24999999515312507}} 

0.5914172919903187 


{'rouge-1': {'r': 0.4166666666666667, 'p': 0.5405405405405406, 'f': 0.4705882303778547}, 'rouge-2': {'r': 0.28, 'p': 0.3888888888888889, 'f': 0.3255813904813413}, 'rouge-l': {'r': 0.2916666666666667, 'p': 0.3783783783783784, 'f': 0.32941175978961945}} 

0.6952201924319762 


{'rouge-1': {'r': 0.358974358974359, 'p': 0.45161290322580644, 'f': 0.39999999506530615}, 'rouge-2': {'r': 0.2926829268292683, 'p': 

{'rouge-1': {'r': 0.2777777777777778, 'p': 0.29411764705882354, 'f': 0.2857142807183674}, 'rouge-2': {'r': 0.16666666666666666, 'p': 0.18181818181818182, 'f': 0.1739130384877128}, 'rouge-l': {'r': 0.1388888888888889, 'p': 0.14705882352941177, 'f': 0.14285713786122467}} 

0.5420471580410908 


{'rouge-1': {'r': 0.08823529411764706, 'p': 0.1875, 'f': 0.11999999564800015}, 'rouge-2': {'r': 0.05714285714285714, 'p': 0.13333333333333333, 'f': 0.07999999580000022}, 'rouge-l': {'r': 0.08823529411764706, 'p': 0.1875, 'f': 0.11999999564800015}} 

0.32988417317187374 


{'rouge-1': {'r': 0.2926829268292683, 'p': 0.42857142857142855, 'f': 0.3478260821340055}, 'rouge-2': {'r': 0.1951219512195122, 'p': 0.25806451612903225, 'f': 0.22222221731867295}, 'rouge-l': {'r': 0.2682926829268293, 'p': 0.39285714285714285, 'f': 0.31884057488762874}} 

0.6254866070065243 


{'rouge-1': {'r': 0.11235955056179775, 'p': 0.4, 'f': 0.17543859306709764}, 'rouge-2': {'r': 0.07446808510638298, 'p': 0.2916666666666667, 

{'rouge-1': {'r': 0.058823529411764705, 'p': 0.09375, 'f': 0.07228915188851823}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0392156862745098, 'p': 0.0625, 'f': 0.04819276634634972}} 

0.5176845710509699 


{'rouge-1': {'r': 0.5185185185185185, 'p': 0.3783783783783784, 'f': 0.43749999512207033}, 'rouge-2': {'r': 0.38461538461538464, 'p': 0.2631578947368421, 'f': 0.3124999951757813}, 'rouge-l': {'r': 0.4444444444444444, 'p': 0.32432432432432434, 'f': 0.37499999512207033}} 

0.4116563427937296 


{'rouge-1': {'r': 0.24096385542168675, 'p': 0.8333333333333334, 'f': 0.3738317722211547}, 'rouge-2': {'r': 0.17708333333333334, 'p': 0.7391304347826086, 'f': 0.2857142825958619}, 'rouge-l': {'r': 0.1566265060240964, 'p': 0.5416666666666666, 'f': 0.2429906507258276}} 

0.7505782241577362 


{'rouge-1': {'r': 0.7368421052631579, 'p': 0.9655172413793104, 'f': 0.8358208906126086}, 'rouge-2': {'r': 0.6153846153846154, 'p': 0.8275862068965517, 'f': 0.705882348049308}, 'rouge-l': {'r'

{'rouge-1': {'r': 0.375, 'p': 0.34615384615384615, 'f': 0.359999995008}, 'rouge-2': {'r': 0.12, 'p': 0.11538461538461539, 'f': 0.11764705382545197}, 'rouge-l': {'r': 0.2916666666666667, 'p': 0.2692307692307692, 'f': 0.27999999500800005}} 

0.4930559590865757 


{'rouge-1': {'r': 0.29310344827586204, 'p': 0.5151515151515151, 'f': 0.3736263690037435}, 'rouge-2': {'r': 0.1724137931034483, 'p': 0.30303030303030304, 'f': 0.21978021515758978}, 'rouge-l': {'r': 0.25862068965517243, 'p': 0.45454545454545453, 'f': 0.3296703250476996}} 

0.5809670011365362 


{'rouge-1': {'r': 0.4583333333333333, 'p': 0.3548387096774194, 'f': 0.3999999950809917}, 'rouge-2': {'r': 0.2608695652173913, 'p': 0.2, 'f': 0.2264150894268424}, 'rouge-l': {'r': 0.2916666666666667, 'p': 0.22580645161290322, 'f': 0.2545454496264464}} 

0.3731696362566814 


{'rouge-1': {'r': 0.16666666666666666, 'p': 0.15, 'f': 0.15789473185595584}, 'rouge-2': {'r': 0.10526315789473684, 'p': 0.10526315789473684, 'f': 0.10526315289473707}, '

In [ ]:
result_dataframe

### BERT Extractor

In [114]:
import pickle

train_idx_filepath = pjoin(kor_bart_data_dir, 'train_data_idx.txt')
test_idx_filepath = pjoin(kor_bart_data_dir, 'test_data_idx.txt')

print(train_idx_filepath)
with open(train_idx_filepath, 'rb') as f:
    train_news_ids = pickle.load(f)
    
print(test_idx_filepath)
with open(test_idx_filepath, 'rb') as f:
    test_news_ids = pickle.load(f)

/data/ksb/BART/KoBART-summarization/data/train_data_idx.txt
/data/ksb/BART/KoBART-summarization/data/test_data_idx.txt


In [115]:
valid_news_ids = train_news_ids[-5000:]
train_news_ids = train_news_ids[:-5000]

In [116]:
assert len(train_news_ids) == 25000
assert len(valid_news_ids) == 5000
assert len(test_news_ids) == 5978

In [117]:
kobert_mapping_data = pjoin(kobert_root, 'urls')
kobert_mapping_data

'/data/ksb/BertSum/koBertSum/urls'

In [118]:
bert_train_idx_filepath = pjoin(kobert_mapping_data, 'train_data_idx.txt')
bert_val_idx_filepath = pjoin(kobert_mapping_data, 'valid_data_idx.txt')
bert_test_idx_filepath = pjoin(kobert_mapping_data, 'test_data_idx.txt')

print(bert_train_idx_filepath)
with open(bert_train_idx_filepath, 'wb') as f:
    pickle.dump([idx for idx in train_news_ids], f)
    
print(bert_val_idx_filepath)
with open(bert_val_idx_filepath, 'wb') as f:
    pickle.dump([idx for idx in valid_news_ids], f)
    
print(bert_test_idx_filepath)
with open(bert_test_idx_filepath, 'wb') as f:
    pickle.dump([idx for idx in test_news_ids], f)

/data/ksb/BertSum/koBertSum/urls/train_data_idx.txt
/data/ksb/BertSum/koBertSum/urls/valid_data_idx.txt
/data/ksb/BertSum/koBertSum/urls/test_data_idx.txt


In [46]:
def get_mapping_list(path, mapping_list):
    
    article_list = []
    
    with jsonlines.open(path) as reader:
        for news in reader:
            content = news['article_original']
            idx = news['id']
            
            if not idx in mapping_list:
                continue
                
            abs_summ = news['abstractive']
            ext_summ = news['extractive']

            article = (idx, content, abs_summ, ext_summ)
            article_list.append(article)
        
    return article_list

In [47]:
train_data = pjoin(data_root, 'train.jsonl')
val_data = pjoin(data_root, 'vaild.jsonl')

In [48]:
bert_train_list = get_mapping_list(train_data, train_news_ids)
bert_train_list += get_mapping_list(val_data, train_news_ids)

In [49]:
bert_valid_list = get_mapping_list(train_data, valid_news_ids)
bert_valid_list += get_mapping_list(val_data, valid_news_ids)

In [57]:
bert_test_list = get_mapping_list(train_data, test_news_ids)
bert_test_list += get_mapping_list(val_data, test_news_ids)

In [62]:
assert len(bert_train_list) == 25000
assert len(bert_valid_list) == 5000
assert len(bert_test_list) == 5978

In [65]:
bert_train_list[0][1]

["대유위니아그룹의 위니아딤채가 발효기술과 음식 별 맞춤보관 기능이 돋보이는 2020년형 김치냉장고 '딤채'의 광고 영상을 공개했다고 26일 밝혔다.",
 "광고는 '우리집 딤채를 소개합니다'라는 컨셉으로 '하나의 딤채로 네 대의 딤채' 효과를 누릴 수 있는 사용성을 강조한다.",
 '딤채 한 대가 지원하는 다양한 식재료 저장 기능을 소개하는 것으로, 한겨울 땅속 김장김치와 묵은지, 이유식 재료·샐러드, 빙온숙성 고기 등 성질이 다른 여러 식품을 최적의 조건으로 보관할 수 있음을 보여준다.',
 "2020년형 딤채는 '오리지널 땅속 냉각' 기능과 새롭게 추가된 '이유식 재료·샐러드 보관모드', 업계 최초 '빙온 고기숙성 모드' 등으로 식품에 최적화된 숙성 및 보관 기능을 지원한다.",
 "메인 카피로 소개되는 '하나의 딤채로 네 대의 딤채를'이 강조한 '오리지널 독립냉각'은 4룸 스탠드형 딤채에 각각의 독립된 냉각기가 있어 소비자의 편의를 극대화한 핵심기능이다.",
 '이를 통해 사계절 가전으로 거듭나고 있는 김치냉장고의 편의성을 높여 다양한 라이프 스타일을 누릴 수 있도록 했다.',
 '딤채의 광고는 TV와 위니아딤채 홈페이지, 유튜브 채널, SNS을 통해서도 만나볼 수 있다.',
 '위니아딤채 관계자는 "대한민국 대표 김치냉장고 브랜드로서 26년간 딤채가 지켜온 오리지널 공식을 광고 영상으로 표현하고자 했다"며 "깜짝 등장하는 딤채쿡을 통해 위니아딤채가 전하는 건강한 집밥에 대한 메시지까지 소비자에게 잘 전달되길 바란다"고 말했다.',
 "대유위니아그룹의 위니아딤채가 발효기술과 음식 별 맞춤보관 기능이 돋보이는 2020년형 김치냉장고 '딤채'의 광고 영상을 공개했다."]

In [90]:
bert_train_list[11][2]

'이재명 경기지사에게 당선무효형을 내린 사법부에 범국민대책위원회가 출범하여 무죄판결을 촉구하였다.'

In [71]:
abstrac_inf = '@highlight\n\n'

In [69]:
kobert_data

'/data/ksb/BertSum/koBertSum/ko_raw_stories'

In [98]:
bert_data = bert_train_list + bert_valid_list + bert_test_list

for data in bert_data:
    
    news_id = data[0]
    article = data[1]
    abs_summ = data[2]
        
    article = list(map(lambda line: line + '\n\n',article))
    abs_summ = abstrac_inf + abs_summ
    
    file_name = os.path.join(kobert_data, str(news_id)+".story")    
    with open(file_name, 'w', encoding='utf-8') as f_s:
        f_s.writelines(article)
        f_s.write(abs_summ)

In [99]:
from functools import reduce
with open(pjoin(kobert_data, '334599022.story'), 'r', encoding='utf-8') as f:
    l = f.readlines()
    txt = reduce(lambda x, y: x+y, l)
    print(txt)

대전지방고용노동청이 한화 대전공장 폭발사고를 두고 특별근로감독에 나선 가운데 한화공장의 처벌 수위에 대한 관심이 모아진다.

청와대 국민청원에 책임자 처벌을 요구하는 글이 수차례 게시된 데 이어 대전노동청도 공식입장을 통해 법 위반 사항에 엄중한 잣대를 들이대겠다고 한만큼 가중 처벌 가능성도 점쳐진다.

18일 고용노동부와 대전지방고용노동청 등에 따르면 대전노동청은 이번 한화대전공장에서 발생한 산업재해사고와 관련해 산업안전보건법 위반 여부를 중점적으로 살피는 중이다.

현행 산업안전보건법(산안법) 제 66조의 2에 따르면 안전조치 또는 보건조치를 위반해 근로자를 사망에 이르게 한 자는 7년 이하의 징역 또는 1억 원 이하의 벌금에 처한다고 명시돼 있다.

이중 안전조치 사항인 산안법 제23조 1항은 사업주가 '폭발성, 발화성 및 인화성 물질 등에 의한 위험'을 예방하기 위해 필요한 조치를 해야 한다고 기재돼 있어, 한화 대전공장의 해당 조항 위반 여부가 주목된다.

이와 함께 중대재해가 발생한 사업장 법인과 안전보건관리 책임자에 대한 처벌이 이뤄지는 양벌 규정도 적용될 것으로 보인다.

가중처벌에 대한 가능성도 점쳐진다.

통상 산업안전보건법 위반에 따른 처벌은 벌금형이 대부분이며 가중처벌에 대한 내용도 내년 1월부터 적용된다.

하지만 한화대전공장 폭발사고가 지난해에 이어 1년도 채 되지 않아 재차 발생한 만큼 사안의 엄중성을 따져 가중처벌(징역형)이 내려질 가능성도 있다는 것이다.

대전노동청 관계자는 "특별근로감독을 포함해 해당 사건에 대한 전반적인 수사가 끝나봐야 알겠지만 법률 위반 정도에 따라 가중처벌이 이뤄질 수도 있다"고 밝혔다.

한편 대전노동청은 이날 오전 폭발사고로 근로자 3명이 사망한 한화대전공장에 대한 특별근로감독에 들어갔다.

노동청은 근로감독관과 안전보건공단 전문가 등을 투입해 오는 28일까지 사업장 전체에 대한 안전보건 진단을 실시한다.

앞서 노동청은 지난 14일 사고발생 직후 한화대전공장에 대한 전면 작업중지 명령을 내린 바 있다

### MatchSum

In [ ]:
def get_result_list(src_regex):
    gold, cand, cand_num, src = [], [], [], []

    for file_paths in iglob(src_regex, recursive=False):
        lines = []
        with open(file_paths) as f:
            lines = f.readlines()

        if file_paths.endswith('gold'):
            gold = lines
        elif file_paths.endswith('candidate_sent_num'):
            cand_num = lines
        elif file_paths.endswith('candidate'):
            cand = lines
        elif file_paths.endswith('src'):
            src = lines
        else:
            print(file_paths)
            raise ValueError
    
    return src, gold, cand, cand_num
    

In [ ]:
src, gold, cand, cand_num = get_result_list(join(bert_result, 'candidate_5_*'))

print("gold len : {}".format(len(gold)))
print("src len : {}".format(len(src)))
print("cand len : {}".format(len(cand)))
print("cand_num len : {}".format(len(cand_num)))

In [ ]:
assert len(gold)==len(src) and len(src)==len(cand) and len(cand_num)==len(cand)

In [ ]:
match_json_data = os.path.join(root_dir,'origin')

del_folder(origin_dir)
mkdir_p(origin_dir)

In [ ]:
origin_json_file = os.path.join(origin_dir, 'origin_data.jsonl')

with open(origin_json_file, 'w', encoding='utf-8') as f:
    writer = jsonlines.Writer(f)
    for _gold, _cand, _src in zip(gold, cand, src):

        src_list = _src.split(split_symbol)
        src_list = list(map(lambda txt : txt.replace('\n',''), src_list))

        abs_list = _gold.split(split_symbol)
        abs_list = list(map(lambda txt : txt.replace('\n',''), abs_list))

        d = {"text" : src_list, "summary" : abs_list}

        writer.write(d) 

In [ ]:
index_json_file = os.path.join(origin_dir, 'index.jsonl')

with open(index_json_file, 'w', encoding='utf-8') as f:
    writer = jsonlines.Writer(f)
    for _cand_num in cand_num:
        
        idx_list = _cand_num.split(split_symbol)
        idx_list = [int(idx.replace('\n','')) for idx in idx_list]
        idx_json = {"sent_id" : idx_list}
        
        writer.write(idx_json)

### RefSum

In [119]:
ref_mapping_data = pjoin(kobert_root, 'ref_urls')
ref_mapping_data

'/data/ksb/BertSum/koBertSum/ref_urls'

In [120]:
import pickle

train_idx_filepath = pjoin(kor_bart_data_dir, 'train_data_idx.txt')
test_idx_filepath = pjoin(kor_bart_data_dir, 'test_data_idx.txt')

print(train_idx_filepath)
with open(train_idx_filepath, 'rb') as f:
    train_news_ids = pickle.load(f)
    
print(test_idx_filepath)
with open(test_idx_filepath, 'rb') as f:
    test_news_ids = pickle.load(f)

/data/ksb/BART/KoBART-summarization/data/train_data_idx.txt
/data/ksb/BART/KoBART-summarization/data/test_data_idx.txt


In [121]:
import random

news_ids = train_news_ids + test_news_ids
random.shuffle(news_ids) 

train_news_ids = news_ids[:25000]
valid_news_ids = news_ids[25000:30000]
test_news_ids = news_ids[30000:]

In [123]:
ref_train_idx_filepath = pjoin(ref_mapping_data, 'train_data_idx.txt')
ref_val_idx_filepath = pjoin(ref_mapping_data, 'valid_data_idx.txt')
ref_test_idx_filepath = pjoin(ref_mapping_data, 'test_data_idx.txt')

mkdir_p(ref_mapping_data)
print(ref_train_idx_filepath)
with open(ref_train_idx_filepath, 'wb') as f:
    pickle.dump([idx for idx in train_news_ids], f)
    
print(ref_val_idx_filepath)
with open(ref_val_idx_filepath, 'wb') as f:
    pickle.dump([idx for idx in valid_news_ids], f)
    
print(ref_test_idx_filepath)
with open(ref_test_idx_filepath, 'wb') as f:
    pickle.dump([idx for idx in test_news_ids], f)

/data/ksb/BertSum/koBertSum/ref_urls/train_data_idx.txt
/data/ksb/BertSum/koBertSum/ref_urls/valid_data_idx.txt
/data/ksb/BertSum/koBertSum/ref_urls/test_data_idx.txt
